In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source,'lxml')

# grabs table
table = soup.find("table", class_ = "wikitable sortable").tbody

print('packages imported')

packages imported


In [2]:
rows = []
for tr in table.find_all('tr'):
    
    tds = tr.find_all('td')
    row = []
    
    for td in tds:
        row.append(td.text.rstrip())
    
    
    if (len(row) == 0) or row[1] == 'Not assigned':
        pass
    
    else:
      
      if row[2] == 'Not assigned':
        row[2] = row[1]
      
      rows.append(row)
      

old_df = pd.DataFrame(rows, columns = ['Postcode','Borough','Neighbourhood'])
old_df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [0]:
def get_index(data, postcode):
  for i in range(len(data)):
    if data[i][0] == postcode:
      return i
  return None

In [0]:
#traverse the old dataframe
new_df = []
for i in range(len(old_df)):
  
  code = old_df.iloc[i,0] #code to look for
  index = get_index(new_df,code) #index of the code (if present)
  
  #print(code)
  #print(index)
  
  temp = old_df.iloc[i,:].to_list() #extract row from existing dataframe 
  #print(temp)
  
  if index != None:
    #postcode is already present in the new dataframe
    neigh = temp[2] #get the neighborhood 
    new_df[index][2] = new_df[index][2] + "," + neigh # add neighborhood
    
  else:
    new_df.append(temp)


In [5]:
new_data = pd.DataFrame(new_df, columns = ['PostalCode','Borough','Neighborhood'])
new_data.sort_values(by=['PostalCode'], ascending = True, inplace = True)
new_data = new_data.reset_index().drop(['index'],axis = 1)

new_data.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
new_data.shape

(103, 3)

In [8]:
cordinates = pd.read_csv('https://cocl.us/Geospatial_data')
cordinates.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
cordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
combined = pd.merge(new_data,cordinates, how = 'left', on = 'PostalCode')
combined.shape

(103, 5)

In [11]:
combined.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
